# EDA

**Исходные данные:** .tif спутниковый снимок, .tif маска с геоданными.

**Цель:** обучить модель Unet для сегментации леса на спутниковых снимках

**План выполнения EDA:**
1. Визуализация изображений
2. Получение основной информации о снимках. Размер, геоданные, количество каналов и так далее
3. Разделение снимка и маски на патчи определенного размера. Их визуализация
4. Формирование датасетов train, val, test, создание соответствующих папок

Количество объектов (деревьев):
* *зеленые*: 2762
* *желтые*: 1087
* *красные*: 566
* *всего*: 4415

## Структура данных

In [65]:
import rasterio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2

In [45]:
def create_dirs(base_dir: str) -> None:
    """
    Создает директории под train, test, val выборки, в которых будут храниться патчи

    :param
        base_dir: Базовая директория для патчей
    :return:
        None: ничего
    """
    os.makedirs(base_dir, exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'train/images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'train/labels'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'test/images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'test/labels'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'val/images'), exist_ok=True)
    os.makedirs(os.path.join(base_dir, 'val/labels'), exist_ok=True)

create_dirs('data/')

In [60]:
def get_info(path_to_image: str) -> None:
    """
    Выводит основную информацию об изображении

    :return:
        None: ничего
    """
    with rasterio.open(path_to_image) as src:
        print("✅ Основная информация:")
        print(f"- Ширина: {src.width}")
        print(f"- Высота: {src.height}")
        print(f"- Количество каналов: {src.count}")
        print(f"- Тип данных: {src.dtypes}")  # список по каналам, например ['uint16']
        print(f"- CRS (система координат): {src.crs}")
        print(f"- Transform (геопривязка): {src.transform}")
        print()

        print("🧾 Полные метаданные (src.meta):")
        print(src.meta)

def get_image_array(path_to_image: str) -> np.ndarray:
    """
    Получаем изображение в виде матрицы формы (height, width, channels)

    :return:
        np.ndarray: Матрица
    """
    return np.array(Image.open(path_to_image))

def visualization_image(path_to_image: str) -> None:
    """
    Визуализируем изображение с помощью matplotlib

    :return:
        None: ничего
    """
    rgb = get_image_array(path_to_image)

    plt.figure(figsize=(10, 15))
    plt.imshow(rgb)
    plt.tight_layout()
    plt.axis('off')
    plt.show()

def count_pixels_by_color(path_to_image: str) -> dict:
    """
    Считаем количество пикселей разных цветов на маске

    :return:
        dict: Словарь с количеством пикселей по цветам на маске
    """
    image = cv2.imread(path_to_image)

    color_counts = {
        'red': 0,
        'yellow': 0,
        'green': 0,
        'black': 0
    }

    # Определим цвета в BGR (OpenCV использует BGR)
    colors_bgr = {
        'red':    ([55, 96, 255], [55, 96, 255]),
        'yellow': ([55, 250, 250], [55, 250, 250]),
        'green':  ([83, 179, 36], [83, 179, 36]),
        'black':  ([0, 0, 0], [0, 0, 0])
    }

    for color, (lower, upper) in colors_bgr.items():
        mask = cv2.inRange(image, np.array(lower), np.array(upper))
        count = cv2.countNonZero(mask)
        color_counts[color] = count

    return color_counts

def slice_image(path_to_image: str, size: int) -> list:
    """
    Нарезает исходное изображение на патчи со стороной размера size

    :param
        size: размер стороны патча
    :return:
        None: ничего
    """
    image = get_image_array(path_to_image)

    patches = []

    height, width, channels = image.shape
    h_steps = (height + size - 1) // size
    w_steps = (width + size - 1) // size

    for h in range(h_steps):
        for w in range(w_steps):

            start_h = h * size
            start_w = w * size
            end_h = min(start_h + size, height)
            end_w = min(start_w + size, width)


            patch = image[start_h:end_h, start_w:end_w, :]


            if patch.shape[0] < size or patch.shape[1] < size:
                padded_patch = np.zeros((size, size, channels), dtype=patch.dtype)
                padded_patch[:patch.shape[0], :patch.shape[1], :] = patch
                patch = padded_patch

            patches.append(patch)

    return patches

def save_patches(patches: list, labels: list, split_type: str, base_dir: str, prefix: str) -> None:
    for i, (patch, label) in enumerate(zip(patches, labels)):
        img = Image.fromarray(patch)
        img.save(os.path.join(base_dir, f'{split_type}/images/{prefix}_{i}.png'))

        label_img = Image.fromarray(label)
        label_img.save(os.path.join(base_dir, f'{split_type}/labels/{prefix}_{i}.png'))

In [63]:
path_to_orig = 'data/RGB_Polazna_cut.tif'
path_to_mask = 'data/TIF_original_image_with_mask&geodata.tif'

patches_orig = slice_image(path_to_orig, 256)
patches_mask = slice_image(path_to_mask, 256)

train_images, test_images, train_labels, test_labels = train_test_split(patches_orig, patches_mask, test_size=0.2, random_state=42)
val_images, test_images, val_labels, test_labels = train_test_split(test_images, test_labels, test_size=0.50, random_state=42)

save_patches(train_images, train_labels, 'train', 'data/', 'patch')
save_patches(val_images, val_labels, 'val', 'data/', 'patch')
save_patches(test_images, test_labels, 'test', 'data/', 'patch')

In [64]:
from glob import glob

pattern = 'data/**/*'
files = glob(pattern, recursive=True)

print(files)

['data\\2025-05-17_19-17-25.png', 'data\\PNG_original_image_with_mask.png', 'data\\RGB_Polazna_cut.tif', 'data\\test', 'data\\TIF_original_image_with_mask&geodata.tif', 'data\\train', 'data\\val', 'data\\test\\images', 'data\\test\\labels', 'data\\test\\images\\patch_0.png', 'data\\test\\images\\patch_1.png', 'data\\test\\images\\patch_10.png', 'data\\test\\images\\patch_100.png', 'data\\test\\images\\patch_101.png', 'data\\test\\images\\patch_102.png', 'data\\test\\images\\patch_103.png', 'data\\test\\images\\patch_11.png', 'data\\test\\images\\patch_12.png', 'data\\test\\images\\patch_13.png', 'data\\test\\images\\patch_14.png', 'data\\test\\images\\patch_15.png', 'data\\test\\images\\patch_16.png', 'data\\test\\images\\patch_17.png', 'data\\test\\images\\patch_18.png', 'data\\test\\images\\patch_19.png', 'data\\test\\images\\patch_2.png', 'data\\test\\images\\patch_20.png', 'data\\test\\images\\patch_21.png', 'data\\test\\images\\patch_22.png', 'data\\test\\images\\patch_23.png', 'd

In [62]:
# 55,96,255 -- красный
# 55,250,250 -- желтый
# 83,179,36 -- зеленый

TODO: Дописать для двух изображений. Исходные изображения не добавлять в репозиторий, а положить на яндекс диск и сделать ссылку. README.md для запуска